<a href="https://colab.research.google.com/github/CJRiverFlow/Data_Lit/blob/master/%5BDatalit%5D_Logistic_Regression_Christian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
#from google.colab import files
#files.upload()

In [0]:
#! unzip us-census-data.zip

In [0]:
#Datasets have no headers, just for info
#columns = ['age','workclass','fnlwgt','education','education-num','marital_status',
#          'ocupation','relationship','race','sex','capital-gain','capital-loss',
#          'hours-per-week','native-country','income']


# Functions

In [0]:
def sigmoid(z):
  result = 1/(1+np.exp(-z))
  return result

In [0]:
def predict(features,weights):
  r = np.dot(features,weights)
  return sigmoid(r)

In [0]:
def cost_function(features, labels, weights):
    observations = len(labels)
    predictions = predict(features, weights)
    class1_cost = -labels*np.log(predictions)
    class2_cost = (1-labels)*np.log(1-predictions)
    cost = class1_cost + class2_cost
    cost = cost.sum() / observations
    return cost

In [0]:
def update_weights(features, labels, weights, lr):
    N = len(features)
    predictions = predict(features, weights)
    gradient = np.dot(features.T,  predictions - labels)
    gradient /= N
    gradient *= lr
    weights -= gradient
    return weights

In [0]:
def decision_boundary(prob):
  return 1 if prob >= 0.5 else 0

In [0]:
def accuracy(predicted_labels, actual_labels):
    diff = predicted_labels - actual_labels
    return 1.0 - (float(np.count_nonzero(diff)) / len(diff))

# **Training algorithm**

In [0]:
#c=[]#global variable

In [0]:
def train_with_file(data_file, iters):
  columns = ['age','workclass','fnlwgt','education','education-num','marital_status',
          'ocupation','relationship','race','sex','capital-gain','capital-loss',
          'hours-per-week','native-country','income']

  df_train = pd.read_csv(data_file,header=None,names=columns)
  
  X_train = df_train.iloc[:,:-1]
  y_train = df_train.iloc[:,-1]

  X_train.replace(' ?', np.nan, inplace=True)
  y_train.replace(' ?', np.nan, inplace=True)
  y_train = y_train.apply(lambda x: 1 if x==' >50K' else 0)
  
  X_train['workclass'].fillna(' 0', inplace=True)
  X_train['ocupation'].fillna(' 0', inplace=True)
  X_train['native-country'].fillna(' 0', inplace=True)
  
  numerical = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
  categorical = ['workclass','education','marital_status','ocupation','relationship','race','sex','native-country']
  
  p = dict(keys=numerical)
  
  #CREATING DICTIONARY

  for i in range(len(numerical)):
    a = min(X_train[numerical[i]])
    b = max(X_train[numerical[i]])
    p[numerical[i]]=(a,b)
  
  for j in range(len(numerical)):
    d = numerical[j]
    X_train[d] =  X_train[d].apply(lambda x: (x-p[d][0])/((p[d][1]-p[d][0])))
  
  X_train = pd.get_dummies(X_train)
  global c 
  c = X_train.columns
  #X_train[categorical] = X_train[categorical].apply(lambda x: pd.factorize(x)[0])
  
  #TRAINING LOOP
  weights = np.random.rand(X_train.shape[1])
  cost_history = []
  lr=0.1

  for i in range(iters):
      weights = update_weights(X_train, y_train, weights, lr)

      cost = cost_function(X_train, y_train, weights)
      cost_history.append(cost)

      #if i % 100 == 0:
      #  predictions = predict(X_train,weights)
      #  predictions=list(map(decision_boundary,predictions))
      #  acc = accuracy(predictions,y_train)
      #  print("iter: "+str(i) + " cost: "+str(cost) + " accuracy: "+str(acc))
  
  return weights, p

In [0]:
#%time weights,normalization_params = train_with_file('adult-training.csv', 1000)

CPU times: user 50.3 s, sys: 29.5 s, total: 1min 19s
Wall time: 40.5 s


In [0]:
def classify(data_file, weights, normalization_params):
  columns = ['age','workclass','fnlwgt','education','education-num','marital_status',
        'ocupation','relationship','race','sex','capital-gain','capital-loss',
        'hours-per-week','native-country','income']
  df_test = pd.read_csv(data_file,names=columns,skiprows=1)
  
  X_test = df_test.iloc[:,:-1]
  y_test = df_test.iloc[:,-1]

  X_test.replace(' ?', np.nan, inplace=True)
  y_test.replace(' ?', np.nan, inplace=True)
  y_test = y_test.apply(lambda x: 1 if x==' >50K.' else 0)
  
  X_test['workclass'].fillna(' 0', inplace=True)
  X_test['ocupation'].fillna(' 0', inplace=True)
  X_test['native-country'].fillna(' 0', inplace=True)
  
  numerical = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
  categorical = ['workclass','education','marital_status','ocupation','relationship','race','sex','native-country']
  #X_test[categorical] = X_test[categorical].apply(lambda x: pd.factorize(x)[0])
  
  q = normalization_params
  for j in range(len(numerical)):
    d = numerical[j]
    X_test[d] =  X_test[d].apply(lambda x: (x-q[d][0])/((q[d][1]-q[d][0])))
    
  X_test = pd.get_dummies(X_test)
  missing_cols = set( c ) - set( X_test.columns )
  for w in missing_cols:
      X_test[w] = 0
  X_test = X_test[c]
  
  predictions = predict(X_test,weights)
  predictions = np.array(list(map(decision_boundary,predictions)))
  
  return predictions

In [0]:
#%time labels = classify('adult-test.csv', weights, normalization_params)

CPU times: user 167 ms, sys: 12.4 ms, total: 179 ms
Wall time: 170 ms


In [0]:
#acc = accuracy(labels,y_test)
#acc

In [0]:
# Our grading program will use these functions as follows:
def sample_main():
  weights, normalization_params = train_with_file('adult-training.csv', 1000)
  labels = classify('adult-test.csv', weights, normalization_params)